<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/Gemma_Python_Chat_(GPC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-assistants-with-gemma:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F64148%2F7669720%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240404%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240404T061645Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da886914e40c0cb34745087de0dbc21cda4e3c814bde561617c0673740e502364117e23e5f43029f75dc8dbfe1bc5f1410c765400364f85ed170d06788a2acff56a892da9894fd3af3ef4051e88c91093eda3f21e516a024b33a86295fbe3667de6533b0ad782c53ed14c1a0a0fc5f9c5eea75907717ff211b0459421003d883225cad9cb893b1eb3431e0040446a5729e930203bb2f5f19ccbdd167c2d907b248cab6c0a98c39cff3fe45295ff0b661c27ec99cf133558aa695f1344337992cce2b3c94df68a31bd2341a02d154f59a5648dd415363a85fafba066ad8cfdc9bac7111b4927038bb26e3fb5265999a3ab823f4b5641c35e9769b188869671c542,pythonquestions:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F262%2F726715%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240404%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240404T061645Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1e927f5cf108286d4500752cb64d1956097d02a59ebd6e835727ad1a819410fa10a88a603e79c188d13a5f5b54fdcaa16ec101d823a014bee0ce206c21d66945d16d3520735110412c2b10b15df0efe8d51fd9ec0f0630e9b4489d3b1e148502ce46d7f2f7edf4bae66704acb247c49d02c9036f61b1b477223c15a706b94d54f4309b85f771b32c48db068d67a3e9cf582a69d6e33a14a35fae8d810316832e105fef0b0b5c71020bbba2af7405cb500fd34af3459091839b6efb5695a055ed4effd13f4c677ca2613b903057c64839ca4bcd78fdab85ababc5c30325d38a749416e0f2bfdcb3a7a6c1aafd5bd44d9a5f15b230e02479a2ed0dcbf6622cd098,gemma/transformers/2b/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F6216%2F11384%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240404%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240404T061645Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D57126f0c83f0076b180b2e1341b5eea217ee5b5da42c89c3e39e4344ef45212bf0e8cbe90fcae310d79ac1dd1a22573202b2d3605b873205b5eadaeeddbacb873b40136e9a3b9e2d724d174f2e44843a66844a5f7180ae4e155f2f61d9e92ec1405d0694723d32a8496a5beba2ba50b96811f7eaaf584f7807d07290ccb8bf7d69ac3afb954023d6f44a7cf4abc8933d65c83a1e3135ff35070d19b74dd39d7875ea36a3ca02f326fc0e48db7d6a45faa432e4a2f26ab6965b1ac021ca97e90daef68391df2bb68dc92496d447165f9826eceb42bbd129e728674fca0759c28f0dc72aed5ea89474d7576c9e7b92e6213ab37a77ded83932fdcdd05d8e84144d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/GPC_(Gemma_Python_Chat).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GCP 🤖 Gemma Python Chatbot

![](https://i.ibb.co/8xZNc32/Gemma.png)

The Gemma Python Chatbot 🚀🚀 help you to answer common questions about the 🐍 Python programming language, powered by [Gemma 2B IT](https://blog.google/technology/developers/gemma-open-models/) updated with the Python Enhancement Proposal ([PEPs](https://peps.python.org/)) documentation using a Retrival-Augmented Generation ([RAG](https://blogs.nvidia.com/blog/what-is-retrieval-augmented-generation/)) sponsored by a Chroma vectorial database using the open source embeddings [hkunlp/instructor-large](https://huggingface.co/hkunlp/instructor-large) of 768 entries, orchested by [langchaing](https://python.langchain.com/docs/use_cases/chatbots/) that is a framework that let you use differen models y les code changes

## Requirements

In [ ]:
!pip install chromadb
# !pip install ydata-profiling
!pip install langchain
#!pip install pydantic
!pip install sentence-transformers
!pip install InstructorEmbedding

In [ ]:
##!pip install langchain-core==0.1.31
#!pip install chromadb==0.3.26
#!pip install ydata-profiling==4.6.1
#!pip install langchain==0.0.345
#!pip install pydantic==1.10.14
#!pip install sentence-transformers==2.6.1
#!pip install InstructorEmbedding==1.0.1

In [ ]:
#base
import os
import json
import re
import uuid
import time
import csv
import pandas as pd
from typing import *
# variables
from kaggle_secrets import UserSecretsClient
# model
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
)
# data
from datasets import load_dataset
import requests
from bs4 import BeautifulSoup
# embeddings
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
# vector database
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# Classes

### Questions and answers data

In [ ]:
class pythonQAData:
    """
    Processes data from Questions and Answers CSV files to provide a structured Q&A format.

    Attributes:
        questions_path (str): Path to the Questions CSV file
        answers_path (str): Path to the Answers CSV file
        tags_path (str): Path to the tags CSV file

    Methods:
        load_data(): Loads the CSV data into DataFrames.
        merge(): Cleans, merges, and formats the question and answer data.
        get_formatted_qa(): Returns a list of formatted question-answer strings.
    """

    def __init__(self):
        self.questions_path = '../input/pythonquestions/Questions.csv'
        self.answers_path = '../input/pythonquestions/Answers.csv'
        self.tags_path = '../input/pythonquestions/Tags.csv'
        self.regex = r"<\/?[\w\s]*>"

    def load_data(self):
        """Loads Questions and Answers data from CSV files."""
        df_questions = pd.read_csv(
            self.questions_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Score',
                'Title'
            ]
        )
        df_answers = pd.read_csv(
            self.answers_path,
            encoding="ISO-8859-1",
            usecols=[
                'ParentId',
                'Score',
                'Body'
            ]
        )
        df_tags = pd.read_csv(
            self.tags_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Tag'
            ]
        )
        return df_questions, df_answers, df_tags


    def qa_data(self):
        """Cleans, merges, and formats the question and answer data."""
        df_questions, df_answers, df_tags = self.load_data()
        # Rename
        df_questions.rename(
            columns={
                'Title': 'Question',
                'Score': 'question_score'
            },
            inplace=True
        )
        df_answers.rename(
            columns={
                'Body': 'Answer',
                'ParentId':'Id',
                'Score': 'answer_score'
            },
            inplace=True
        )
        # Filter by score
        df_questions = df_questions[df_questions['question_score'] > 5].copy()
        # Sort and deduplicate answers
        df_answers = df_answers.sort_values(
            'answer_score',
            ascending=False
        ).drop_duplicates(subset=['Id'])
        # Merge
        df_qa = df_questions.merge(
            df_answers,
            how='left',
            on='Id'
        ).merge(
            df_tags,
            how='left',
            on='Id'
        )
        # filter for python  questions
        df_qa = df_qa[df_qa['answer_score'] > 5].copy()
        df_qa = df_qa[df_qa['Tag']=='python'].copy()
        df_qa['Answer'] = df_qa['Answer'].apply(
            lambda x: re.sub(
                self.regex,
                "",
                x
            )
        )
        return df_qa

    def get_fine_tunning_data(self):
        """Returns a list of formatted user-assistant strings."""
        df_qa_data = self.qa_data()
        data = [
            f"<-change-of-interlocutor->user:\n{row['Question']}\n<-change-of-interlocutor->assistant:\n{row['Answer']}"
            for index, row
            in df_qa_data.iterrows()
        ]
        return data
    def get_qa_data(self):
        """Returns a list of records dictionaries """
        df_qa_data = self.qa_data()
        data = df_qa_data[
            [
                'Question',
                'Answer'
            ]
        ].to_dict(orient='records')
        return data


### Python Enhancement Proposals data

In [ ]:
class Peps:
    """
    Scrapes Python Enhancement Proposals (PEPs) from https://peps.python.org/
    """

    def __init__(self):
        """Initiates the scraping process."""
        self.base_url = 'https://peps.python.org/'

    def scraper(self, url):
        """
        Fetches the HTML content of a given URL.

        Args:
            url (str): The URL to fetch.

        Returns:
            BeautifulSoup: A BeautifulSoup object representing the parsed HTML.
        """
        response = requests.get(url)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')

    def _fetch_pep_links(self):
        """Fetches links to individual PEP pages (private method).

        Returns:
            list: A list of PEP URLs.
        """
        soup = self.scraper(self.base_url)
        return list(
            set(
                [
                    self.base_url + ref['href']
                    for ref in soup.find_all('a', class_='pep reference internal')
                ]
            )
        )

    def _download_peps(self):
        """Downloads the content of individual PEPs (private method).

        Returns:
            list: A list of BeautifulSoup objects representing individual PEPs.
        """
        pep_links = self._fetch_pep_links()
        return [self.scraper(pep_link) for pep_link in pep_links]

    def scrape(self):
        """Extracts the relevant text content from each PEP.

        Returns:
            list: A list of text strings, each representing the content of a PEP.
        """
        pep_soups = self._download_peps()
        return [
            '\n'.join([word.text.replace('"""',"'") for word in soup.find_all('section')])
            for soup in pep_soups
        ]

    def jsonl_format(self):
        """Format the spcraped data to a jsonl format is it alist of dictionaries
        Returns:
            list: A list of dictionaries with the following:
                page_content: The content of each section scraped
                source: The linke where it was scraped
        """
        source_links = self._fetch_pep_links()
        pep_content = self.scrape()
        pep_data = dict(zip(source_links,pep_content))
        return [
            {
                'text': value,
                'source': key
            }
            for key, value in pep_data.items()
        ]

### Embeddings

In [ ]:
class Embeder:
    """
    Creates embeddings (numerical representations) of text documents for various tasks like semantic search and
    similarity comparison. Provides flexibility to choose between a large model for more comprehensive
    embeddings or a smaller, faster model.

    Attributes:

    large (bool): Flag indicating whether to use the large embedding model (default: True).
    model (str): Name of the embedding model to use.
    device (str): Device to use for computations (default: "cuda").
    """
    def __init__(
            self,
            large: bool = False
    ):
        """
        init(self, large: bool = True) -> None
            Initializes the class with the specified model size preference.
        """
        self.large = large
        if self.large:
            self.model = "hkunlp/instructor-large"
            self.device: str = "cuda"
        else:
            self.model = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

    def instructor(self):
        if self.large:
            embed = HuggingFaceInstructEmbeddings(
                model_name=self.model,
                model_kwargs={"device": self.device}
            )
        else:
            embed = SentenceTransformerEmbeddings(
                model_name=self.model
            )
        return embed

    def run(
            self,
            docs_list: list,
    ):
        """
        run(self, docs_list: list) -> List[List[float]]:
            Generates embeddings for the provided list of documents.
            Returns a list of lists, where each inner list represents the embedding vector for a document.
        """
        if docs_list is None:
            docs_list = ['']
        embed = self.instructor()

        return embed.embed_documents(docs_list)

### Prompt

In [ ]:
prompt_config = [{
    "prompt_id": "0.0.1",
    "supplier": "google",
    "system": """
Act as a Python programming language expert assistant
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
        """,
    "technical_documentation": "\n\nTake into account this technical documentation found:\n{technical_documentation}",
    "model": "/kaggle/input/gemma/transformers/2b/2",
    "temperature": 0,
    "max_length": 4096,
    "max_tokens": 830,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
}]

### Settings

In [ ]:
class Settings():
    def __init__(self):
        self.user_secrets = UserSecretsClient()
        self.CHUNK_SIZE = self.user_secrets.get_secret("CHUNK_SIZE")
        self.CHUNK_OVERLAP = self.user_secrets.get_secret("CHUNK_OVERLAP")
        self.CHROMA_NAME_INDEX = self.user_secrets.get_secret("CHROMA_NAME_INDEX")
        self.K = self.user_secrets.get_secret("K")
        self.NN_THRESHOLD = self.user_secrets.get_secret("NN_THRESHOLD")
        self.PROMPT_ID = self.user_secrets.get_secret("PROMPT_ID")
        self.prompt = [prompt for prompt in prompt_config if prompt['prompt_id']==self.PROMPT_ID][0]

### Retriever

In [ ]:
embeder = Embeder()
settings = Settings()
class Retriever:
    """ retriever
    Retrieves the embeddings from a vectorial database
    """
    def __init__(self, data: list[dict]):
        """ initialize the retriever
        """
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=int(settings.CHUNK_SIZE),
            length_function=len,
            chunk_overlap=int(settings.CHUNK_OVERLAP)
        )
        self.index_name = settings.CHROMA_NAME_INDEX
        self.data = data

    def load(self):
        """
        Loads documents from the initialized data and returns them as a list of Document objects.
        """
        documents = []
        for obj in self.data:
            page_content = obj.get("text", "")
            metadata = {
                "source": obj.get("source", "local")
            }
            documents.append(Document(page_content=page_content, metadata=metadata))
        return documents

    def set(self):
        """
         update the vectorial database
        """
        data = self.load()
        splitter = self.text_splitter
        documents = splitter.split_documents(data)
        instructor = embeder.instructor()
        vector_db = Chroma.from_documents(
            documents=documents,
            embedding=instructor,
            persist_directory=self.index_name
        )
        vector_db.persist()

    def query(
            self,
            message: str,
            k: int = int(settings.K),
            threshold: float = float(settings.NN_THRESHOLD)
    ):
        """ retrieve the 2 more similar text chunks based on the message given
        """
        vectorial_db = Chroma(
            embedding_function=embeder.instructor(),
            persist_directory=self.index_name
        )
        res = vectorial_db.similarity_search_with_score(message, k=k)
        return [vector[0].page_content for vector in res if vector[1] < threshold]

### Message

In [ ]:
settings = Settings()
class Message:
    def __init__(
            self,
            role: str,
            content: str,
            timestamp: str = int(time.time()),
            prompt_id: str = settings.PROMPT_ID
    ):
        self.reply_id = str(uuid.uuid4())
        self.role = role
        self.content = content
        self.timestamp = timestamp
        self.file = 'data/history.json'
        if not os.path.exists(self.file):
            json.dump([], open(self.file, 'w'))

    def reply(self):
        return {
            'reply_id': self.reply_id,
            'role': self.role,
            'content': self.content,
            'timestamp': self.timestamp
        }

    def system_reply(self):
        prompt = settings.prompt
        return {
            'reply_id': self.reply_id,
            'role': 'system',
            'content': prompt['system'],
            'timestamp': self.timestamp
        }
    def new_chat(self):
        init_chat = [self.system_reply, self.reply]
        json.dump(init_chat, open(self.file, 'w'))
        return init_chat
    def update(self):
        dict_history = json.load(open(self.file))
        dict_history.append(self.reply())
        json.dump(dict_history, open(self.file, 'w'))
    def restart_history(self):
        json.dump([], open(self.file, 'w'))

### Model Gemma

In [ ]:
class Gemma:
    def __init__(self):
        self.prompt = settings.prompt
        self.model_checkpoint = self.prompt['model']
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_checkpoint)
        self.gemma = AutoModelForCausalLM.from_pretrained(
            self.model_checkpoint,
            torch_dtype=torch.float16
        ).cuda()
    def chat(self, context: str):
        # Use the model
        input_text = context
        input_ids = self.tokenizer(input_text, return_tensors="pt")
        input_ids = {k: v.to("cuda") for k, v in input_ids.items()}  if torch.cuda.is_available() else {k: v.to("cpu")}
        outputs = self.gemma.generate(**input_ids, max_length=self.prompt['max_length'])
        return self.tokenizer.decode(outputs[0])

### Expert Python Agent/Assistant

In [ ]:
retriever = Retriever(data)
gemma = Gemma()

class Agent():
    def __init__(self):
        self.prompt = settings.prompt
        self.file = 'data/history.json'
        if not os.path.exists(self.file):
            json.dump([], open(self.file, 'w'))
    def augmented_question(self, question: str):
        technical_documentation = retriever.query(question)
        if len(technical_documentation)>0:
            docs = self.prompt['technical_documentation']
            technical_documentation = '\n* '.join(technical_documentation)
            technical_documentation = technical_documentation[0]
            docs = docs.format(technical_documentation=technical_documentation)
            augmented_reply = f"""{question}{docs}"""
            return augmented_reply
        else:
            return question
    def memory(self, question: str):
        dict_history = json.load(open(self.file))
        message = Message(
                role='user',
                content=self.augmented_question(question)
            )
        if len(dict_history)>0:
            message.update()
            full_chat = json.load(open(self.file))
        else:
            full_chat = message.new_chat()
        return '\n'.join(
            [
                '\n<-change-of-interlocutor->'+reply['role']+': '+reply['content']
                for reply
                in full_chat
            ]
        )+'\n<-change-of-interlocutor->assistant:'
    def chat(self, question: str):
        memory = self.memory(question)
        full_answer = gemma.chat(context=memory)
        print(full_answer)
        answer_list = full_answer.split('<-change-of-interlocutor->')
        pos_list = [
            pos
            for
            pos, answer
            in enumerate(answer_list)
            if question in answer
        ]
        answer = answer_list[pos_list[0]+1]
        return answer


## Data

### fine-tunning data

In [ ]:
%%time
PythonQAData = pythonQAData()
python_qa = PythonQAData.get_qa_data()
print(python_qa[1486])

In [ ]:
%%time
PythonQAData = pythonQAData()
python_context = PythonQAData.get_fine_tunning_data()
print(
    'Python Questions loaded:',
    len(python_context),
    '\n',
    '\nSample question-answer:\n',
    python_context[-1][:1000]
)

## few-shot data

In [ ]:
few_shot_list=[7,6009,12,36,8034,130,141,537,1057,5042]
for i, q in enumerate(few_shot_list):
    print('_'*100,f'\n{i}.',python_qa[q])

## RAG data

In [ ]:
%%time
peps = Peps()
peps_corpus = peps.scrape()
print(
    'corpus lenght:',
    len(peps_corpus),
    '\nsection <n> lenght:',
    len(peps_corpus[0]),
    '\nsample text:\n\n',
    peps_corpus[10][1306:2000]
)

In [ ]:
%%time
peps = Peps()
data = peps.jsonl_format()
data[-1]['text'][:2000]

## RAG

In [ ]:
%%time
embeder = Embeder()
vec = embeder.run(["hello I'm goku"])[0]
print(len(vec))
vec[:10]

In [ ]:
%%time
retriever = Retriever(data)
retriever.set()

In [ ]:
retriever = Retriever(data)
question = """
Version Control System
Currently the CPython and supporting repositories use Mercurial. As a modern distributed version control system, it has served us well since the migration from Subversion. However, when evaluating the VCS we must consider the capabilities of the VCS itself as well as the network effect and mindshare of the community around that VCS.

There are really only two real options for this, Mercurial and Git. The technical capabilities of the two systems are largely equivalent, therefore this PEP instead focuses on their social aspects.

It is not possible to get exact numbers for the number of projects or people which are using a particular VCS, however we can infer this by looking at several sources of information for what VCS projects are using.

The Open Hub (previously Ohloh) statistics [1] show that 37% of the repositories indexed by The Open Hub are using Git (second only to Subversion which has 48%) while Mercurial has just 2%,
Does the community advantage of Git outweigh the familiarity of Mercurial for CPython?
"""
retriever.query(question)[1]

## Chatbot

## Loading few-shot to the memory

In [ ]:
file = 'data/history.json'
init_chat = [message.system_reply()]
json.dump(init_chat, open(file, 'w'))
for i, q in enumerate(few_shot_list):
    sample = python_qa[q]
    mesage = Message(
        role='user',
        content=sample['Question']
    )
    reply = mesage.update()
    mesage = Message(
        role='assitant',
        content=sample['Answer']
    )
    reply = mesage.update()

history = json.load(open(file))
history

### Testing the RAG system

In [ ]:
agent = Agent()
print(agent.memory("""
Version Control System
Currently the CPython and supporting repositories use Mercurial. As a modern distributed version control system, it has served us well since the migration from Subversion. However, when evaluating the VCS we must consider the capabilities of the VCS itself as well as the network effect and mindshare of the community around that VCS.

There are really only two real options for this, Mercurial and Git. The technical capabilities of the two systems are largely equivalent, therefore this PEP instead focuses on their social aspects.

It is not possible to get exact numbers for the number of projects or people which are using a particular VCS, however we can infer this by looking at several sources of information for what VCS projects are using.

The Open Hub (previously Ohloh) statistics [1] show that 37% of the repositories indexed by The Open Hub are using Git (second only to Subversion which has 48%) while Mercurial has just 2%,
"""))

## GPC - 0.0.1

In [ ]:
question

In [ ]:
%%time
agent = Agent()
print('*'*2000,agent.chat('what is the lates python version that is currently lunched'))

## Fine-Tunning

## GPC - 0.1.0